## Prueba tecnica Ingeniero de Datos Pragma
Presentado por: Nicolas Restrepo Carvajal

* Ingeniero de Datos, MsC en Ingeniería , Docente universitario

In [6]:
import sqlite3
import os
import time 
import csv
import os
import time 
import pandas as pd


In [7]:
def remove_temp_files(directory):
    for filename in os.listdir(directory):
        file_path = os.path.join(directory, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)  # This removes directories
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))
    
def replace_missing_values(input_folder, output_folder):
    files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]
    files = sorted(files, key=lambda x: int(x.split('-')[1].split('.')[0]))

    for file in files:
        in_file_path = os.path.join(input_folder, file)
        out_file_path=os.path.join(output_folder, file)
        dataset_name = os.path.splitext(file)[0]
        print(in_file_path)
        print(out_file_path)

        print(dataset_name)
        with open(in_file_path, 'r') as infile, open(out_file_path, 'w', newline='') as outfile:
            reader = csv.reader(infile)
            writer = csv.writer(outfile)
            for row in reader:
                # Replace empty strings with '0'
                row = [value if value.strip() else '0' for value in row]
                writer.writerow(row)

def connect_sqlite(path):
    conn = sqlite3.connect(path)
    cursor = conn.cursor()
    return conn
    
def close_sqlite(conn):
    conn.close()

def execute(statement):
    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()
    query = statement
    cursor.execute(statement)
    conn.commit()
    rows = cursor.fetchall()

    for row in rows:
        print(row)
    conn.close()

   

def insert_csv_line_sqlite(directory_path,sqlite_db_path,table_name):
    conn = sqlite3.connect(sqlite_db_path)
    cursor = conn.cursor()
    files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]
    files = sorted(files, key=lambda x: int(x.split('-')[1].split('.')[0]))
    log_row_count=0
    logs_prices=[]
    for file in files:

        file_path = os.path.join(directory_path, file)
        dataset_name = os.path.splitext(file)[0]
        print(file_path)
        print(dataset_name)
        
        with open(file_path, newline='', encoding='utf-8') as csvfile:
            csv_reader = csv.reader(csvfile)
            next(csv_reader)
            
            for row in csv_reader:
                query = f'INSERT INTO {table_name} (timestamp, price, user_id) VALUES (?, ?, ?)'
                log_row_count+=1
                logs_prices.append(int(row[1]))  
                print(f"--------------------- row {log_row_count} start---------------------\nStats:\nInserted rows count :  {log_row_count}\nAverage of Price: {sum(logs_prices) /len(logs_prices)}\nMinimun Price: {min(logs_prices)}\nMax Price : {max(logs_prices)}\n--------------------- row {log_row_count} end---------------------\n>>>")
                cursor.execute(query, row)
                time.sleep(1.5)
        conn.commit()
    conn.close()

In [ ]:

if __name__ == "__main__":
    raw_data_path = "../Data/raw"
    temp_path="../Data/temp"
    replace_missing_values(raw_data_path,temp_path)
    sqlite_db_path='../database/pragma.db'
    table_name='prices'
    drop_table=f"""drop table if exists {table_name}"""
    execute(drop_table)
    create_table = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        timestamp DATE,
        price double precision,
        user_id INTEGER
    );
    """
    execute(create_table)
    insert_csv_line_sqlite(temp_path,sqlite_db_path,table_name)
    time.sleep(3)
    remove_temp_files(temp_path)
    select_table=f"""select count(1) from {table_name}"""
    execute(select_table)
    


../Data/raw/2012-1.csv
../Data/temp/2012-1.csv
2012-1
../Data/raw/2012-2.csv
../Data/temp/2012-2.csv
2012-2
../Data/raw/2012-3.csv
../Data/temp/2012-3.csv
2012-3
../Data/raw/2012-4.csv
../Data/temp/2012-4.csv
2012-4
../Data/raw/2012-5.csv
../Data/temp/2012-5.csv
2012-5
../Data/temp/2012-5.csv
2012-5
--------------------- row 1 start---------------------
Stats:
Inserted rows count :  1
Average of Price: 82.0
Minimun Price: 82
Max Price : 82
--------------------- row 1 end---------------------
>>>
--------------------- row 2 start---------------------
Stats:
Inserted rows count :  2
Average of Price: 71.5
Minimun Price: 61
Max Price : 82
--------------------- row 2 end---------------------
>>>
--------------------- row 3 start---------------------
Stats:
Inserted rows count :  3
Average of Price: 54.0
Minimun Price: 19
Max Price : 82
--------------------- row 3 end---------------------
>>>
--------------------- row 4 start---------------------
Stats:
Inserted rows count :  4
Average of P

In [54]:
select_price_avg=f"""select avg(price) from {table_name}"""
execute(select_price_avg)

(56.26573426573427,)


In [55]:
select_row_count=f"""select count(1)  from {table_name}"""
execute(select_row_count)

(143,)


In [56]:
select_max_price_=f"""select max(price) from {table_name}"""
execute(select_max_price_)

(100.0,)


In [57]:
select_min_price=f"""select min(price) from {table_name}"""
execute(select_min_price)

(0.0,)


In [9]:
select_min_price=f"""select * from {table_name}"""
execute(select_min_price)

('5/1/2012', 82.0, 7)
('5/2/2012', 61.0, 1)
('5/3/2012', 19.0, 3)
('5/4/2012', 94.0, 3)
('5/5/2012', 17.0, 5)
('5/6/2012', 19.0, 3)
('5/7/2012', 66.0, 6)
('5/8/2012', 13.0, 7)
('5/9/2012', 31.0, 6)
('5/10/2012', 95.0, 6)
('5/11/2012', 77.0, 7)
('5/12/2012', 16.0, 4)
('5/13/2012', 22.0, 8)
('5/14/2012', 91.0, 4)
('5/15/2012', 44.0, 2)
('5/16/2012', 84.0, 9)
('5/17/2012', 64.0, 3)
('5/18/2012', 83.0, 7)
('5/19/2012', 30.0, 7)
('5/20/2012', 62.0, 8)
('5/21/2012', 89.0, 9)
('5/22/2012', 88.0, 1)
('5/23/2012', 34.0, 10)
('5/24/2012', 100.0, 2)
('5/25/2012', 77.0, 9)
('5/26/2012', 91.0, 3)
('5/27/2012', 61.0, 2)
('5/28/2012', 67.0, 8)
('5/29/2012', 39.0, 10)
('5/30/2012', 57.0, 8)
('5/31/2012', 33.0, 6)
('1/10/2012', 50.0, 9)
('1/11/2012', 87.0, 10)
('1/12/2012', 64.0, 7)
('1/13/2012', 20.0, 10)
('1/14/2012', 14.0, 10)
('1/15/2012', 95.0, 8)
('1/16/2012', 95.0, 2)
('1/17/2012', 62.0, 4)
('1/18/2012', 46.0, 8)
('1/19/2012', 97.0, 2)
('1/20/2012', 31.0, 1)
('1/21/2012', 0.0, 2)
('1/22/2012', 3

In [59]:
#testing

In [3]:
files = [f for f in os.listdir("../Data/raw") if f.endswith('.csv')]

In [4]:
print(files)

['2012-5.csv', '2012-1.csv', '2012-3.csv', '2012-4.csv', '2012-2.csv']


In [5]:
files = sorted(files, key=lambda x: int(x.split('-')[1].split('.')[0]))

# Printing the sorted list
print(sorted_files)

['2012-1.csv', '2012-2.csv', '2012-3.csv', '2012-4.csv', '2012-5.csv']
